In [1]:
from lsst.rsp import get_tap_service
import sqlite3
import pandas as pd

In [2]:
service = get_tap_service("tap")

In [98]:
db_fname = "adler_demo_testing_database_dp1.db"

In [99]:
cnx = sqlite3.connect(db_fname)

In [6]:
object_choice_query = f"""
SELECT * FROM dp1.SSObject ORDER BY numObs DESC
"""

object_choice_table = service.search(object_choice_query).to_table().to_pandas()

In [7]:
object_choice_table

,discoverySubmissionDate,numObs,ssObjectId
0,NaN,133,21163620217073748
1,60853.579024,82,23133931615301680
2,NaN,80,21163637482928473
3,NaN,66,21164728252512342
4,60853.579024,65,23133931615301681
...,...,...,...
426,NaN,1,21163646070699585
427,NaN,1,21163611409953348
428,NaN,1,21164741153143117
429,NaN,1,21165810483998787


In [76]:
# ssoid = 8268570668335894776
ssoid_list = (21163620217073748, 23133931615301680, 21163637482928473, 21164728252512342, 23133931615301681)

test_id = 23133931615301680
# 2^63 - 1 is the maximum signed 64-bit value that can be stored in SQLite hence only two extra zeros for the fake_id otherwise it's too big
# max__ = 9223372036854775807
fake_id = 2313393161530168000

In [77]:
filter_name = "r"

In [78]:
# diasource_sql_query = f"""
#             SELECT
#                 *
#             FROM
#                 dp03_catalogs_10yr.diaSource
#             WHERE
#                 ssObjectId = {ssoid}
#             """
diasource_sql_query = f"""
            SELECT
                *
            FROM
                dp1.DiaSource
            WHERE
                ssObjectId in {ssoid_list}
            """

In [79]:
diatable = service.search(diasource_sql_query).to_table().to_pandas()

In [80]:
sssource_sql_query = f"""
            SELECT
                *
            FROM
                dp1.SSSource
            WHERE
                ssObjectId in {ssoid_list}
            """

In [81]:
sssource_table = service.search(sssource_sql_query).to_table().to_pandas()

In [82]:
sssource_table.columns

Index(['diaSourceId', 'eclipticBeta', 'eclipticLambda', 'galacticB',
       'galacticL', 'heliocentricDist', 'heliocentricVX', 'heliocentricVY',
       'heliocentricVZ', 'heliocentricX', 'heliocentricY', 'heliocentricZ',
       'phaseAngle', 'residualDec', 'residualRa', 'ssObjectId',
       'topocentricDist', 'topocentricVX', 'topocentricVY', 'topocentricVZ',
       'topocentricX', 'topocentricY', 'topocentricZ'],
      dtype='object')

In [83]:
ssobject_sql_query = f"""
            SELECT
                *
            FROM
                dp1.SSObject
            WHERE
                ssObjectId in {ssoid_list}
            """

In [84]:
ssobject_table = service.search(ssobject_sql_query).to_table().to_pandas()

In [85]:
ssobject_table.columns

Index(['discoverySubmissionDate', 'numObs', 'ssObjectId'], dtype='object')

In [86]:
mpcorb_sql_query = f"""
            SELECT
                *
            FROM
                dp1.MPCORB
            WHERE
                ssObjectId in {ssoid_list}
            """

In [87]:
mpcorb_table = service.search(mpcorb_sql_query).to_table().to_pandas()

In [88]:
mpcorb_table

,e,epoch,incl,mpcDesignation,mpcH,node,peri,q,ssObjectId,t_p
0,0.094924,60600.0,26.629507,2003 QT30,17.618999,162.155730,152.135140,1.709361,21163620217073748,60398.283691
1,0.369674,60800.0,26.628340,2007 VY289,15.862000,141.890622,293.978022,1.998627,21163637482928473,60696.346311
2,0.197105,60200.0,34.916978,2015 MV118,17.476000,114.148669,173.996577,2.086360,21164728252512342,58695.334369
3,0.109862,60800.0,22.242227,2024 WH103,20.798000,172.747117,247.467015,1.602490,23133931615301680,60643.947618
4,0.058592,60800.0,24.350102,2024 WJ103,19.969999,154.626201,294.724974,1.813405,23133931615301681,60722.281421


In [24]:
# add a new fake object

In [89]:
diatable[(diatable["ssObjectId"] == test_id)].value_counts("diaSourceId")

diaSourceId
600412541927030826    1
600412550919618634    1
600412551456489536    1
600416933392678955    1
600416933526896680    1
                     ..
600430129011228678    1
600438909667639328    1
600438910875598903    1
600438915707437084    1
600438918526009370    1
Name: count, Length: 82, dtype: int64

In [90]:
diatable[(diatable["ssObjectId"] == test_id) & (diatable["band"] == "r")].value_counts("diaSourceId")

diaSourceId
600412541927030826    1
600412551456489536    1
600416933392678955    1
600416933526896680    1
600416933795332129    1
600421328622190602    1
600421328756408378    1
600421328890626113    1
600421329024843806    1
600421329157488679    1
600421331976585269    1
600421332110278677    1
600421332244496435    1
600421332781367370    1
600421332917157900    1
600421333049802798    1
600421336003117058    1
600421336539988015    1
600421336675254292    1
600421336809472060    1
600421336943689808    1
600421337077907497    1
600421337210552359    1
600430121225027640    1
600430121359245328    1
600430121629253714    1
600430124446253090    1
600430124714688548    1
600430124984696869    1
600430125116817423    1
600430125251035143    1
600430125654212635    1
600430128474357787    1
600430128875438111    1
600430129011228678    1
Name: count, dtype: int64

In [91]:
df_dict = {"dia":diatable,
           "sss":sssource_table,
           "sso":ssobject_table,
           "mpc":mpcorb_table}

In [92]:
for x in df_dict:

    _df = df_dict[x].copy()[df_dict[x]["ssObjectId"]==test_id]
    _df["ssObjectId"] = fake_id
    df_dict[x] = pd.concat([df_dict[x],_df],axis = 0).reset_index(drop=True)
    print(len(_df),len(df_dict[x]))

82 508
82 508
1 6
1 6


In [93]:
diatable = df_dict["dia"]
sssource_table = df_dict["sss"]
ssobject_table = df_dict["sso"]
mpcorb_table = df_dict["mpc"]

In [94]:
_df = diatable.copy()[diatable["ssObjectId"]==test_id]
_df["ssObjectId"] = fake_id
diatable = pd.concat([diatable,_df],axis = 0).reset_index(drop=True)
# print(len(_df),len(df_dict[x]))
_df

,apFlux,apFlux_flag,apFlux_flag_apertureTruncated,apFluxErr,band,bboxSize,centroid_flag,coord_dec,coord_ra,dec,...,trailAngle,trailDec,trailFlux,trailLength,trailRa,visit,x,xErr,y,yErr
0,1969.300049,False,False,305.600006,g,22,False,-27.691702,52.653511,-27.691702,...,60.321084,-27.691694,1503.500000,1.029816,52.653506,2024120500124,3702.188110,0.574045,3127.817813,0.790955
1,1503.699951,False,False,306.531006,g,22,False,-27.692430,52.655248,-27.692430,...,-99.260368,-27.692427,1584.550049,1.101433,52.655253,2024120500103,3446.518132,0.332736,2692.015618,0.375375
2,2761.500000,False,False,625.004028,i,17,False,-27.693802,52.658398,-27.693802,...,-78.691434,-27.693802,3654.030029,0.746342,52.658400,2024120500067,3041.719219,0.180834,1917.039886,0.206893
3,2052.719971,False,False,296.699005,g,24,False,-27.694117,52.659175,-27.694117,...,-70.288969,-27.694112,1809.640015,1.274469,52.659177,2024120500058,3669.129356,0.354257,3212.436630,0.369847
13,2992.560059,False,False,445.723999,r,21,False,-27.883677,53.697689,-27.883677,...,-5.422717,-27.883678,3244.179932,1.053293,53.697685,2024112900282,3836.830731,0.282088,3641.023348,0.230282
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381,2298.439941,False,False,386.937988,r,20,False,-27.872731,53.354563,-27.872731,...,-96.323402,-27.872733,2835.100098,0.909912,53.354561,2024120100205,652.693080,0.160580,1758.339559,0.206403
382,2612.629883,False,False,303.739014,g,22,False,-27.873275,53.361761,-27.873266,...,137.486555,-27.873266,2236.290039,0.782020,53.361759,2024120100148,2701.412858,0.188645,2771.620986,0.189725
383,3226.600098,False,False,308.476013,g,21,False,-27.873275,53.361761,-27.873270,...,148.178998,-27.873267,2585.469971,0.658797,53.361665,2024120100149,1519.930587,0.175664,3547.920566,0.177505
384,4148.009766,False,False,398.278015,r,25,False,-27.873318,53.362377,-27.873296,...,-143.778510,-27.873297,3707.020020,0.731660,53.362191,2024120100145,944.595492,0.207874,1222.993291,0.182137


In [95]:
import astropy.units as u
import numpy as np
def flux_to_magnitude(flux, flux_err=np.nan):
    """Converts a flux measurement (with units of nanoJanskys) and its associated error
    into AB magnitudes. If no flux error is provided, the returned magnitude error
    will be NaN.

    Parameters
    -----------
    flux : astropy.units.Quantity
        Flux value with units of nanoJanskys (u.nJy).

    flux_err : astropy.units.Quantity, optional
        Flux error with units of nanoJanskys (u.nJy). Default is np.nan (dimensionless),
        in which case the magnitude error will be returned as NaN.

    Returns
    -----------
    magnitude : float
        The flux converted into AB magnitude (unitless scalar).

    magnitude_err : float
        The propagated uncertainty in AB magnitude (unitless scalar).
        Returns NaN if flux_err is not provided.

    Notes
    -----------
    - This function assumes that `flux` and `flux_err` are `astropy.units.Quantity`
      objects in units of `u.nJy`.
    """
    # TODO Handle the masked arrays better here
    # (ideally I think we want to keep magnitude as a masked array rather than making magErr non-masked)
    magnitude = flux.to(u.ABmag).value
    magnitude_err = ((2.5 / np.log(10)) * (flux_err / flux)).value
    return magnitude, magnitude_err

def add_outburst_fluxspace(flux, mag_shift = -1.5):
    """
    Function for adding a synthetic outburst to a light curve in flux space as the DP1 DiaSource table does not contain magnitude.
    Converts the flux values to magnitude, applies the magnitude shift and then converts back to flux
    
    Parameters
    -----------
    flux : float or array-like
        The flux value(s) to apply the shift to
    mag_shift : float, optional
        The value in magnitude to shift the data by. Default: -1.5

    Returns
    -----------
    output_flux : float or array-like
        The flux value(s) with the magnitude shift applied
    """
    mag, *_ = flux_to_magnitude(flux * u.nJy)
    mag += mag_shift
    output_flux = (mag * u.ABmag).to(u.nJy).value
    return output_flux

In [96]:
# add an outburst
t0 = 60648
t1 = 60653
mag_shift = -1.5

mask = ((diatable["ssObjectId"] == fake_id) &
        (diatable["midpointMjdTai"] > t0) &
        (diatable["midpointMjdTai"] < t1))

# DP1 DiaSource table does not contain magnitude so have to add the outburst in flux space
diatable.loc[mask,"apFlux"] = add_outburst_fluxspace(diatable.loc[mask, "apFlux"].to_numpy(), mag_shift = mag_shift)
diatable.loc[mask,"psfFlux"] = add_outburst_fluxspace(diatable.loc[mask, "psfFlux"].to_numpy(), mag_shift = mag_shift)
diatable.loc[mask,"scienceFlux"] = add_outburst_fluxspace(diatable.loc[mask, "scienceFlux"].to_numpy(), mag_shift = mag_shift)
diatable.loc[mask,"trailFlux"] = add_outburst_fluxspace(diatable.loc[mask, "trailFlux"].to_numpy(), mag_shift = mag_shift)

In [100]:
# diatable.to_sql("DiaSource", con=cnx, if_exists="append", index=False)
diatable.to_sql("DiaSource", con=cnx, if_exists="replace", index=False)

590

In [101]:
# sssource_table.to_sql("ssSource", con=cnx, if_exists="append", index=False)
sssource_table.to_sql("SSSource", con=cnx, if_exists="replace", index=False)

508

In [102]:
# ssobject_table.to_sql("ssObject", con=cnx, if_exists="append", index=False)
ssobject_table.to_sql("SSObject", con=cnx, if_exists="replace", index=False)

6

In [103]:
# mpcorb_table.to_sql("MPCORB", con=cnx, if_exists="append", index=False)
mpcorb_table.to_sql("MPCORB", con=cnx, if_exists="replace", index=False)

6

In [ ]:
# _diatable = diatable.copy()[diatable["ssObjectId"] == test_id]
# _diatable["ssObjectId"] = fake_id
# _diatable["diaSourceId"] = _diatable["diaSourceId"] * 1000

# # add an outburst
# t0 = 63100
# t1 = 63600
# mag_shift = -1.5

# mask = (
#     (_diatable["ssObjectId"] == fake_id)
#     & (_diatable["midPointMjdTai"] > t0)
#     & (_diatable["midPointMjdTai"] < t1)
# )

# _diatable.loc[mask, "mag"] += mag_shift

# _diatable.to_sql("diaSource", con=cnx, if_exists="append", index=False)

In [ ]:
# _diatable[diatable["band"] == "r"]

In [ ]:
# _sssource_table = sssource_table.copy()[sssource_table["ssObjectId"] == test_id]
# _sssource_table["ssObjectId"] = fake_id
# _sssource_table["diaSourceId"] = _sssource_table["diaSourceId"] * 1000
# _sssource_table.to_sql("ssSource", con=cnx, if_exists="append", index=False)

# _ssobject_table = ssobject_table.copy()[ssobject_table["ssObjectId"] == test_id]
# _ssobject_table["ssObjectId"] = fake_id
# _ssobject_table.to_sql("ssObject", con=cnx, if_exists="append", index=False)

# _mpcorb_table = mpcorb_table.copy()[mpcorb_table["ssObjectId"] == test_id]
# _mpcorb_table["ssObjectId"] = fake_id
# _mpcorb_table.to_sql("MPCORB", con=cnx, if_exists="append", index=False)

In [ ]:
# cnx.close()

Testing everything went correctly...

In [ ]:
# cnx = sqlite3.connect(db_fname)

In [ ]:
# # example_query = f"""
# #                 SELECT
# #                     ssObject.ssObjectId, mag, magErr, band, midpointMjdTai, ra, dec, phaseAngle,
# #                     topocentricDist, heliocentricDist
# #                 FROM
# #                     ssObject
# #                     JOIN diaSource ON ssObject.ssObjectId   = diaSource.ssObjectId
# #                     JOIN ssSource  ON diaSource.diaSourceId = ssSource.diaSourceId
# #                 WHERE
# #                     ssObject.ssObjectId = {ssoid}
# #                 """
# # example_query = f"""
# #                 SELECT
# #                     ssObject.ssObjectId, mag, magErr, band, midpointMjdTai, ra, dec, phaseAngle,
# #                     topocentricDist, heliocentricDist
# #                 FROM
# #                     ssObject
# #                     JOIN diaSource ON ssObject.ssObjectId   = diaSource.ssObjectId
# #                     JOIN ssSource  ON diaSource.diaSourceId = ssSource.diaSourceId
# #                 WHERE
# #                     ssObject.ssObjectId in {ssoid_list}
# #                 """

# example_query = f"""
#                 SELECT
#                     ssObject.ssObjectId, ssSource.diaSourceId, mag, magErr, band, midpointMjdTai, ra, dec, phaseAngle,
#                     topocentricDist, heliocentricDist
#                 FROM
#                     ssObject
#                     JOIN diaSource ON ssObject.ssObjectId   = diaSource.ssObjectId
#                     JOIN ssSource  ON diaSource.diaSourceId = ssSource.diaSourceId
#                 WHERE
#                     ssObject.ssObjectId = {fake_id} and band = 'r'
#                 """

In [ ]:
# pd.read_sql_query(example_query, cnx)

In [ ]:
# cur = cnx.cursor()

In [ ]:
# res = cur.execute("SELECT * FROM sqlite_schema")

In [ ]:
# res.fetchall()